<a href="https://colab.research.google.com/github/christianloring/Projects/blob/main/multivariate_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings
from scipy.stats.stats import ttest_ind

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'studentsperformance'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

students = pd.read_sql_query('select * from studentsperformance',con=engine)

engine.dispose()

In [ ]:
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


All columns filled, don't need to fill any data. Check if data is categorical or numerical.

In [ ]:
students.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Are there any differences between the genders, ethnicities, and parental level of education with respect to student performances in exams?

Categorical: Gender, race, parental education, lunch, test prep

Numerical: Math/reading/writing scores

In [ ]:
students.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

In [ ]:
gen_score_table = students.groupby("gender").mean()[["math score", "reading score", "writing score"]]

In [ ]:
gen_score_table

,math score,reading score,writing score
gender,,,
female,63.633205,72.608108,72.467181
male,68.728216,65.473029,63.311203


girls higher reading/writing, boys higher math. is it due to random chance? 

In [ ]:
ttest_ind(students[students.gender == "female"]["math score"],
          students[students.gender == "male"]["math score"])

Ttest_indResult(statistic=-5.383245869828983, pvalue=9.120185549328822e-08)

In [ ]:
ttest_ind(students[students.gender == "female"]["writing score"],
          students[students.gender == "male"]["writing score"])

Ttest_indResult(statistic=9.979557910004507, pvalue=2.019877706867934e-22)

In [ ]:
ttest_ind(students[students.gender == "female"]["reading score"],
          students[students.gender == "male"]["reading score"])

Ttest_indResult(statistic=7.959308005187657, pvalue=4.680538743933289e-15)

all p values significantly less than 0.05, so we can say the difference is significant

In [ ]:
eth_score_table = students.groupby("race/ethnicity").mean()[["math score", "reading score", "writing score"]]
par_score_table = students.groupby("parental level of education").mean()[["math score", "reading score", "writing score"]]

In [ ]:
eth = students["race/ethnicity"].unique

In [ ]:
eth

<bound method Series.unique of 0      group B
1      group C
2      group B
3      group A
4      group C
        ...   
995    group E
996    group C
997    group C
998    group D
999    group D
Name: race/ethnicity, Length: 1000, dtype: object>

In [ ]:
eth = students["race/ethnicity"].unique()
for subject in ["math score", "writing score", "reading score"]:
    print("\n\n{}".format(subject))
    for i in range(0, len(eth)):
        for j in range(i, len(eth)):
            print(
                "t-test between groups {0} and {1}:".format(eth[i], eth[j]))
            print(ttest_ind(
                students[students["race/ethnicity"] == eth[i]][subject],
                students[students["race/ethnicity"] == eth[j]][subject]
            ))

READING: CD, BA, BC

WRITING: DE, BA, BC

MATH: BC, BA, CA

all of these have significant differences according to p values.

In [ ]:
students.groupby("parental level of education").mean()[["math score", "reading score", "writing score"]]

,math score,reading score,writing score
parental level of education,,,
associate's degree,67.882883,70.927928,69.896396
bachelor's degree,69.389831,73.000000,73.381356
high school,62.137755,64.704082,62.448980
master's degree,69.745763,75.372881,75.677966
some college,67.128319,69.460177,68.840708
some high school,63.497207,66.938547,64.888268


In [ ]:
education = students["parental level of education"].unique()

In [ ]:
for subject in ["math score", "reading score", "writing score"]:
  print("\n\n{}".format(subject))
  for i in range(0, len(education)):
      for j in range(i, len(education)):
          print(
              "t-test between groups {0} and {1}:".format(education[i], education[j]))
          print(ttest_ind(
                students[students["parental level of education"] == education[i]][subject],
                students[students["parental level of education"] == education[j]][subject]
            ))




math score
t-test between groups bachelor's degree and bachelor's degree:
Ttest_indResult(statistic=0.0, pvalue=1.0)
t-test between groups bachelor's degree and some college:
Ttest_indResult(statistic=1.3702352829617477, pvalue=0.17151224218471559)
t-test between groups bachelor's degree and master's degree:
Ttest_indResult(statistic=-0.14868196568849312, pvalue=0.8819757824867702)
t-test between groups bachelor's degree and associate's degree:
Ttest_indResult(statistic=0.8786641889793243, pvalue=0.38020757818036177)
t-test between groups bachelor's degree and high school:
Ttest_indResult(statistic=4.236143622484822, pvalue=2.9965345853843376e-05)
t-test between groups bachelor's degree and some high school:
Ttest_indResult(statistic=3.196718602676573, pvalue=0.0015409192922812146)
t-test between groups some college and some college:
Ttest_indResult(statistic=0.0, pvalue=1.0)
t-test between groups some college and master's degree:
Ttest_indResult(statistic=-1.2356334215824571, pvalue

There are many p values > 0.05, indicating that this is a very important factor. 

In [ ]:
students.groupby("lunch").mean()[["math score", "reading score", "writing score"]]

,math score,reading score,writing score
lunch,,,
free/reduced,58.921127,64.653521,63.022535
standard,70.034109,71.654264,70.823256


visually there seems to be a difference

In [ ]:

ttest_ind(students[students.lunch == "free/reduced"][["math score", "reading score", "writing score"]],
          students[students.lunch == "standard"][["math score", "reading score", "writing score"]])

Ttest_indResult(statistic=array([-11.83718047,  -7.45105647,  -8.0097842 ]), pvalue=array([2.41319560e-30, 2.00279665e-13, 3.18618958e-15]))

p value confirms, could be linked to other variables though. For example, people who have higher degrees earn more money and can better educate their kids, so those with better parental education will be less likely to have reduced/free lunches. 

In [ ]:
students.groupby("test preparation course").mean()[["math score", "reading score", "writing score"]]

,math score,reading score,writing score
test preparation course,,,
completed,69.695531,73.893855,74.418994
none,64.077882,66.534268,64.504673


In [ ]:
ttest_ind(students[students["test preparation course"] == "completed"][["math score", "reading score", "writing score"]],
          students[students["test preparation course"] == "none"][["math score", "reading score", "writing score"]])

Ttest_indResult(statistic=array([ 5.70461642,  7.87166354, 10.40917344]), pvalue=array([1.53591346e-08, 9.08178334e-15, 3.68529174e-24]))

Yes, p values are incredibly small. We can say there is significant evidence those who complete the course are more successful.

In [ ]:
students[["math score", "reading score", "writing score"]].corr()

,math score,reading score,writing score
math score,1.000000,0.817580,0.802642
reading score,0.817580,1.000000,0.954598
writing score,0.802642,0.954598,1.000000


Reading and writing have a correlation of 0.954598, being the highest among all score correlations. 